<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=98fcb18b4e04af6424dd6681c63afde2822c632929729c971167311e22af0a4e
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-12 11:22:10
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  4.95 L
Current:  1.40 C
-------------------
Today PnL: 2.08 K (0.01%)
Current PnL: -28.62 L (-18.49%)
CY Booked + Current PnL: -14.33 L (-9.26%)
-------------------
Total profit:  1.28 L
Total loss:  -29.90 L
-------------------
Total Booked + Current PnL: 12.65 L (9.89%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.23%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.33 C
Est FTT PnL: 93.26 L (66.78%)
Deployed:  1.28 C
Current:  1.40 C
CAGR/XIRR %: 4.95%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,0.04,-16.77,20.27,0.09,17001.0,-16904.0,83873.0,94.66,42.0,M-SC,2.37,253.0,-0.99,0.62,10.19,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,-0.57,-12.97,14.93,0.02,22680.0,-22647.0,151912.0,131.58,51.0,M-SC,15.79,234.0,-1.00,1.13,32.14,OX40N,NTT,PAINTS
50,MASFIN,398.61,0.48,-1.56,23.98,22.05,23129.0,-1530.0,96450.0,-15.09,57.0,H-SC,6.94,164.0,-0.07,0.72,40.20,XR,ATH,FINANCE
43,ITC,452.00,-0.58,-2.13,12.84,10.44,25150.0,-4269.0,195869.0,-41.56,42.0,X-LC,1.84,5.0,-0.17,1.45,3.31,X40,NTT,FMCG
82,VOLTAS,1530.00,0.54,6.45,12.44,19.69,25391.0,12363.0,204105.0,-3.98,51.0,X-MC,2.58,78.0,0.49,1.52,13.68,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,DIXON,18931.72,2.71,-14.33,41.92,21.58,50329.0,-20079.0,120060.0,-56.81,36.0,X-MC,7.72,56.0,-0.40,0.89,8.01,X40N,ATH,IT
6,ASIANTILES,137.00,2.27,-13.99,111.22,81.67,89739.0,-13124.0,80686.0,7255.56,53.0,L-SC,13.68,271.0,-0.15,0.60,56.25,XR,NTT,CERAMICS
48,KPIGREEN,730.66,2.10,-17.84,78.86,46.95,81180.0,-22355.0,102942.0,-24.08,36.0,H-SC,6.88,154.0,-0.28,0.76,28.96,MH,ATH,POWER
10,BANDHANBNK,400.00,1.85,-28.81,170.31,92.42,337713.0,-80266.0,198293.0,39.37,48.0,H-SC,8.77,170.0,-0.24,1.47,14.60,XY24,NTT,BANKS
46,JSWINFRA,342.00,1.57,-8.63,26.11,15.23,47797.0,-17287.0,183060.0,-21.89,44.0,X-MC,8.89,66.0,-0.36,1.36,20.84,X40N,NTT,REALTY


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,HINDUNILVR,2922.0,-1.93,-12.04,29.23,13.67,66089.0,-30963.0,226100.0,-18.61,25.0,X-LC,2.42,14.0,-0.47,1.68,6.74,XY25,NTT,FMCG
27,GLAXO,3466.2,-1.55,3.63,34.27,39.15,70774.0,7236.0,206520.0,-16.74,53.0,X-MC,11.05,60.0,0.10,1.53,33.69,X40,ATH,PHARMA
44,JCHAC,2282.0,-1.32,-37.62,60.30,-0.01,48072.0,-48083.0,79722.0,16204.35,22.0,M-SC,5.90,233.0,-1.00,0.59,2.04,OX40N,NTT,AC
64,SHALBY,327.0,-1.21,-25.95,68.95,25.10,92483.0,-47013.0,134130.0,885.65,28.0,M-SC,10.81,250.0,-0.51,1.00,7.49,XY24,NTT,HEALTHCARE
57,RAJOOENG,143.1,-1.03,-37.37,122.90,39.61,78902.0,-38300.0,64200.0,-57.57,24.0,H-SC,28.04,135.0,-0.49,0.48,0.00,AR,ATH,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.28,-6.72,122.90,107.91,169253.0,-9928.0,137716.0,-24.20,39.0,M-SC,12.51,216.0,-0.06,1.02,2.68,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-0.57,-12.97,14.93,0.02,22680.0,-22647.0,151912.0,131.58,51.0,M-SC,15.79,234.0,-1.00,1.13,32.14,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,0.04,-16.77,20.27,0.09,17001.0,-16904.0,83873.0,94.66,42.0,M-SC,2.37,253.0,-0.99,0.62,10.19,OX40N,NTT,DURABLES
66,SIS,528.00,0.34,-25.25,62.21,21.26,51636.0,-28030.0,83002.0,1956.99,44.0,H-SC,4.77,166.0,-0.54,0.62,12.14,OX40N,NTT,MISC
47,KANSAINER,340.00,-0.14,-25.59,52.51,13.47,105359.0,-69021.0,200646.0,-69.48,30.0,H-SC,4.71,158.0,-0.66,1.49,2.02,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,-0.06,6.27,62.1,72.27,111827.0,10627.0,180075.0,-8.8,66.0,M-LC,2.01,99.0,0.1,1.34,12.83,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-0.06,6.27,62.10,72.27,111827.0,10627.0,180075.0,-8.80,66.0,M-LC,2.01,99.0,0.10,1.34,12.83,XR,NTT,IT
38,INDIAMART,4810.62,-0.77,-4.53,116.54,106.72,137220.0,-5591.0,117745.0,-52.73,30.0,H-SC,2.33,139.0,-0.04,0.87,17.78,AR,ATH,MISC
85,ZYDUSLIFE,1286.17,0.59,-2.14,38.60,35.63,79164.0,-4486.0,205088.0,-15.85,42.0,H-MC,3.11,119.0,-0.06,1.52,14.17,AR,ATH,PHARMA
25,FINCABLES,1641.55,-0.28,-6.72,122.90,107.91,169253.0,-9928.0,137716.0,-24.20,39.0,M-SC,12.51,216.0,-0.06,1.02,2.68,OX40N,ATH,CABLES
37,IEX,219.00,-0.23,-3.55,54.10,48.63,104336.0,-7097.0,192857.0,-35.77,50.0,H-SC,14.04,136.0,-0.07,1.43,7.59,XR,NTT,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.28,-21.85,89.95,48.45,140953.0,-43804.0,156702.0,-27.39,24.0,X-MC,12.71,64.0,-0.31,1.16,2.03,X40N,ATH,FINANCE
33,HINDUNILVR,2922.00,-1.93,-12.04,29.23,13.67,66089.0,-30963.0,226100.0,-18.61,25.0,X-LC,2.42,14.0,-0.47,1.68,6.74,XY25,NTT,FMCG
75,TMPV,600.00,-0.29,-18.09,73.59,42.18,166608.0,-50009.0,226401.0,-26.74,29.0,X-LC,6.40,3.0,-0.30,1.68,0.66,XY24,NTT,AUTO
23,DMART,5391.45,-0.08,-9.04,40.85,28.12,78183.0,-19017.0,191390.0,-21.49,32.0,X-LC,5.23,19.0,-0.24,1.42,12.45,X40N,ATH,FMCG
53,PAGEIND,51605.07,-0.08,-9.53,39.34,26.07,58278.0,-15600.0,148140.0,-33.39,33.0,X-MC,12.39,55.0,-0.27,1.10,0.93,X40,ATH,APPARELS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,4762.0,0.33,-80.10,527.24,24.79,264184.0,-201749.0,50107.0,-81.09,46.0,X-SC,0.96,86.0,-0.76,0.37,22.49,X40N,NTT,MISC
43,ITC,452.0,-0.58,-2.13,12.84,10.44,25150.0,-4269.0,195869.0,-41.56,42.0,X-LC,1.84,5.0,-0.17,1.45,3.31,X40,NTT,FMCG
8,AWL,485.0,0.46,-21.34,95.13,53.50,225805.0,-64386.0,237365.0,-61.97,35.0,X-MC,2.09,58.0,-0.29,1.76,5.43,XY24,NTT,FMCG
36,ICICIPRULI,790.0,0.46,5.89,23.68,30.97,44318.0,10416.0,187154.0,-18.97,61.0,X-MC,2.10,75.0,0.24,1.39,19.19,X40,ATH,INSURANCE
3,ACC,3906.0,0.47,-24.89,118.55,64.15,211873.0,-59231.0,178720.0,-55.88,37.0,X-SC,2.25,82.0,-0.28,1.33,0.47,XY24,BTT,CEMENT


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,PGHH,17907.65,-0.05,-6.21,42.62,33.76,80271.0,-12480.0,188340.0,-33.85,33.0,X-MC,5.37,57.0,-0.16,1.40,0.00,X40,ATH,FMCG
3,ACC,3906.00,0.47,-24.89,118.55,64.15,211873.0,-59231.0,178720.0,-55.88,37.0,X-SC,2.25,82.0,-0.28,1.33,0.47,XY24,BTT,CEMENT
75,TMPV,600.00,-0.29,-18.09,73.59,42.18,166608.0,-50009.0,226401.0,-26.74,29.0,X-LC,6.40,3.0,-0.30,1.68,0.66,XY24,NTT,AUTO
53,PAGEIND,51605.07,-0.08,-9.53,39.34,26.07,58278.0,-15600.0,148140.0,-33.39,33.0,X-MC,12.39,55.0,-0.27,1.10,0.93,X40,ATH,APPARELS
31,HAVELLS,2069.16,0.24,-9.99,47.19,32.48,139313.0,-32773.0,295218.0,-14.48,40.0,X-MC,5.69,67.0,-0.24,2.19,1.14,X40,ATH,ELECTRICAL


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-0.39,-30.12,104.43,42.84,47435.0,-19583.0,45423.0,-54.27,40.0,X-SC,36.21,83.0,-0.41,0.34,2.76,XY24,NTT,MISC
15,CAMS,4762.00,0.33,-80.10,527.24,24.79,264184.0,-201749.0,50107.0,-81.09,46.0,X-SC,0.96,86.0,-0.76,0.37,22.49,X40N,NTT,MISC
59,RELAXO,1176.00,-0.25,-47.46,189.87,52.31,144818.0,-68888.0,76272.0,-44.05,43.0,X-SC,6.65,91.0,-0.48,0.57,2.09,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.01,-38.07,118.33,35.20,94285.0,-48990.0,79680.0,4.40,34.0,X-SC,15.89,92.0,-0.52,0.59,1.21,X40,NTT,FOOTWEAR
22,DIXON,18931.72,2.71,-14.33,41.92,21.58,50329.0,-20079.0,120060.0,-56.81,36.0,X-MC,7.72,56.0,-0.40,0.89,8.01,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,0.26,-13.05,37.18,19.28,111845.0,-45137.0,300819.0,-24.27,61.0,X-LC,4.94,1.0,-0.40,2.23,11.21,X40,ATH,IT
41,INFY,2275.00,-0.49,8.25,43.06,54.87,145850.0,25825.0,338713.0,-15.65,60.0,X-LC,3.30,2.0,0.18,2.51,17.22,X40,BTT,IT
75,TMPV,600.00,-0.29,-18.09,73.59,42.18,166608.0,-50009.0,226401.0,-26.74,29.0,X-LC,6.40,3.0,-0.30,1.68,0.66,XY24,NTT,AUTO
81,VBL,671.64,-0.64,-3.96,41.26,35.67,125157.0,-12505.0,303337.0,-15.30,55.0,X-LC,4.45,4.0,-0.10,2.25,9.35,X40N,ATH,FMCG
43,ITC,452.00,-0.58,-2.13,12.84,10.44,25150.0,-4269.0,195869.0,-41.56,42.0,X-LC,1.84,5.0,-0.17,1.45,3.31,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,0.72,-37.87,113.89,32.89,54631.0,-29238.0,47968.0,-692.31,53.0,L-MC,5.84,259.0,-0.54,0.36,32.10,XR,NTT,BANKS
6,ASIANTILES,137.00,2.27,-13.99,111.22,81.67,89739.0,-13124.0,80686.0,7255.56,53.0,L-SC,13.68,271.0,-0.15,0.60,56.25,XR,NTT,CERAMICS
50,MASFIN,398.61,0.48,-1.56,23.98,22.05,23129.0,-1530.0,96450.0,-15.09,57.0,H-SC,6.94,164.0,-0.07,0.72,40.20,XR,ATH,FINANCE
13,BSOFT,831.70,0.90,-21.72,92.61,50.79,101972.0,-30544.0,110109.0,0.50,68.0,H-SC,5.98,151.0,-0.30,0.82,28.20,XR,ATH,IT
39,INDIGOPNTS,1408.00,-0.57,-12.97,14.93,0.02,22680.0,-22647.0,151912.0,131.58,51.0,M-SC,15.79,234.0,-1.00,1.13,32.14,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.7,0.90,-21.72,92.61,50.79,101972.0,-30544.0,110109.0,0.50,68.0,H-SC,5.98,151.0,-0.30,0.82,28.20,XR,ATH,IT
1,ABB,7934.0,0.39,-3.44,50.75,45.57,128207.0,-8995.0,252624.0,-37.97,59.0,H-MC,4.78,120.0,-0.07,1.88,7.49,AR,NTT,ELECTRICAL
56,RAJESHEXPO,518.0,-0.23,-62.91,169.81,0.07,86394.0,-86300.0,50877.0,1764.90,57.0,L-SC,2.42,268.0,-1.00,0.38,26.53,OX40N,NTT,JEWELLERY
0,5PAISA,593.0,-0.29,-41.63,93.41,12.90,119141.0,-90950.0,127546.0,87.50,42.0,H-SC,11.85,161.0,-0.76,0.95,6.05,OX40N,NTT,FINANCE
84,WIPRO,420.0,-0.06,6.27,62.10,72.27,111827.0,10627.0,180075.0,-8.80,66.0,M-LC,2.01,99.0,0.10,1.34,12.83,XR,NTT,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.76
1,25,45.66
2,50,76.64


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.14
MC    30.30
LC    24.60
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.32
X40      22.35
X40N     10.87
XR       10.19
XY25      9.39
AR        9.36
OX40N     7.77
SR        1.03
MH        0.76
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.63
X-MC    23.52
X-LC    21.02
M-SC    12.10
X-SC     6.95
H-MC     5.02
L-SC     1.46
M-MC     1.40
M-LC     1.34
H-LC     1.18
L-LC     1.06
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.31,-7.94,42.85
IT,12.79,-16.90,80.11
FINANCE,9.61,-19.88,70.53
ELECTRICAL,6.26,-11.36,52.15
MISC,6.09,-58.51,123.44
PAINTS,6.04,-13.61,30.49
INSURANCE,4.60,-2.10,38.08
PHARMA,4.09,-1.26,33.67
AUTO,3.42,-24.82,78.35


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3223676.0,21
AR,1309694.0,10
XR,1292519.0,13
X40,1026728.0,14
X40N,983373.0,9
OX40N,746989.0,10
XY25,378706.0,6
SR,282789.0,2
MH,81180.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3663668.0,25
M-SC,1428985.0,15
X-MC,1314002.0,16
X-SC,985776.0,8
X-LC,929851.0,11
H-MC,402391.0,3
L-SC,265604.0,3
M-LC,111827.0,1
H-LC,74551.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1279450.0      6
           AR         900660.0      5
M-SC       XY24       814016.0      6
H-SC       XR         773006.0      7
X-SC       X40N       506445.0      3
X-MC       X40        458081.0      7
           XY24       399657.0      3
X-LC       X40        384140.0      5
H-SC       OX40N      346583.0      4
M-SC       OX40N      314012.0      5
X-SC       XY24       294824.0      3
H-SC       SR         282789.0      2
X-MC       X40N       273588.0      4
X-LC       XY24       240709.0      2
H-MC       AR         207371.0      2
X-LC       X40N       203340.0      2
H-MC       XY24       195020.0      1
X-SC       X40        184507.0      2
X-MC       XY25       182676.0      2
L-SC       XR         179210.0      2
M-SC       XR         173845.0      2
           AR         127112.0      2
M-LC       XR         111827.0      1
X-LC       XY25       101662.0      2
L-SC       OX40N       86394.0      1
H-SC       MH          81180.0      1
H-LC       AR          74551.0      1
M-MC       XY25        57260.0      1
L-MC       XR          54631.0      1
L-LC       XY25        37108.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 26.0 seconds
